In [21]:
import numpy as np
import pandas as pd
import requests
import re
from tqdm import tqdm #progress bar library -- not necessary, but helpful since some of this is slower

In [22]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 150)
%matplotlib inline

## Gather Historical Player Data

In [31]:
df = pd.DataFrame()

for season in ['2017-18', '2018-19', '2019-20', '2020-21']:
    season_df = pd.read_csv(f'/Users/andrew.peters/Documents/GitHub/Fantasy-Premier-League/data/{season}/gws/merged_gw.csv', encoding = 'ISO-8859-1')
    season_df['season'] = season
    df = df.append(season_df, ignore_index=True)

In [38]:
#the columns aren't the same across all seasons
df.groupby('season').count()

,name,assists,attempted_passes,big_chances_created,big_chances_missed,bonus,bps,clean_sheets,clearances_blocks_interceptions,completed_passes,creativity,dribbles,ea_index,element,errors_leading_to_goal,errors_leading_to_goal_attempt,fixture,fouls,goals_conceded,goals_scored,ict_index,id,influence,key_passes,kickoff_time,kickoff_time_formatted,loaned_in,loaned_out,minutes,offside,open_play_crosses,opponent_team,own_goals,penalties_conceded,penalties_missed,penalties_saved,recoveries,red_cards,round,saves,selected,tackled,tackles,target_missed,team_a_score,team_h_score,threat,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,winning_goals,yellow_cards,GW,position,team,xP
season,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2017-18,22467,22467,22467,22467,22467,22467,22467,22467,22467,22467,22467,22467,22467,22467,22467,22467,22467,22467,22467,22467,22467,22467,22467,22467,22467,22467,22467,22467,22467,22467,22467,22467,22467,22467,22467,22467,22467,22467,22467,22467,22467,22467,22467,22467,22467,22467,22467,22467,22467,22467,22467,22467,22467,22467,22467,22467,0,0,0
2018-19,21790,21790,21790,21790,21790,21790,21790,21790,21790,21790,21790,21790,21790,21790,21790,21790,21790,21790,21790,21790,21790,21790,21790,21790,21790,21790,21790,21790,21790,21790,21790,21790,21790,21790,21790,21790,21790,21790,21790,21790,21790,21790,21790,21790,21790,21790,21790,21790,21790,21790,21790,21790,21790,21790,21790,21790,0,0,0
2019-20,22560,22560,0,0,0,22560,22560,22560,0,0,22560,0,0,22560,0,0,22560,0,22560,22560,22560,0,22560,0,22560,0,0,0,22560,0,0,22560,22560,0,22560,22560,0,22560,22560,22560,22560,0,0,0,22501,22501,22560,22560,22560,22560,22560,22560,22560,0,22560,22560,0,0,0
2020-21,24365,24365,0,0,0,24365,24365,24365,0,0,24365,0,0,24365,0,0,24365,0,24365,24365,24365,0,24365,0,24365,0,0,0,24365,0,0,24365,24365,0,24365,24365,0,24365,24365,24365,24365,0,0,0,24365,24365,24365,24365,24365,24365,24365,24365,24365,0,24365,24365,24365,24365,24365


In [39]:
# some of the early seasons had lots of extraneous columns we don't need -- only keep the columns that were used in the most reason season
df = df[list(season_df)]

In [40]:
df.isna().sum()

name                     0
position             66817
team                 66817
xP                   66817
assists                  0
bonus                    0
bps                      0
clean_sheets             0
creativity               0
element                  0
fixture                  0
goals_conceded           0
goals_scored             0
ict_index                0
influence                0
kickoff_time             0
minutes                  0
opponent_team            0
own_goals                0
penalties_missed         0
penalties_saved          0
red_cards                0
round                    0
saves                    0
selected                 0
team_a_score            59
team_h_score            59
threat                   0
total_points             0
transfers_balance        0
transfers_in             0
transfers_out            0
value                    0
was_home                 0
yellow_cards             0
GW                       0
season                   0
d

In [41]:
columns_to_drop = [
    'xP',
    'own_goals',
    'penalties_missed',
    'penalties_saved',
    'red_cards',
    'saves',
    'selected',
    'threat',
    'transfers_balance',
    'transfers_in',
    'transfers_out',
    'yellow_cards'
]

In [42]:
df = df.drop(columns=columns_to_drop)

In [48]:
df.columns = df.columns.str.lower()

In [49]:
df.tail()

,name,position,team,assists,bonus,bps,clean_sheets,creativity,element,fixture,goals_conceded,goals_scored,ict_index,influence,kickoff_time,minutes,opponent_team,round,team_a_score,team_h_score,total_points,value,was_home,gw,season
91177,Aaron Ramsdale,GK,Sheffield Utd,0,2,30,1,10.0,483,376,0,0,3.2,22.0,2021-05-23T15:00:00Z,90,4,38,0.0,1.0,9,46,True,38,2020-21
91178,Adrien Silva,MID,Leicester,0,0,0,0,0.0,220,373,0,0,0.0,0.0,2021-05-23T15:00:00Z,0,17,38,4.0,2.0,0,44,True,38,2020-21
91179,Sidnei Tavares,MID,Leicester,0,0,0,0,0.0,676,373,0,0,0.0,0.0,2021-05-23T15:00:00Z,0,17,38,4.0,2.0,0,44,True,38,2020-21
91180,Kalvin Phillips,MID,Leeds,0,0,17,0,12.8,204,372,1,1,6.1,40.6,2021-05-23T15:00:00Z,90,18,38,1.0,3.0,6,49,True,38,2020-21
91181,Raphael Dias Belloli,MID,Leeds,1,0,24,0,60.2,570,372,1,0,12.9,37.4,2021-05-23T15:00:00Z,90,18,38,1.0,3.0,5,56,True,38,2020-21


In [50]:
df.to_pickle('raw_data.pkl')

## Gather Historical Fixtures Data

Idea for fixture difficulty rating:
- scrape FbRef or Understat for G, GA, xG, xGA, xGD
- can build out a fixture-difficulty rating (or two, one attacing, one defense), based on figures above over previous ten games
- back test to see how well it lined up